In [1]:
from IslandTime import TimeSeriesConnections, retrieve_island_info, plot_shoreline_transects
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2


Successfully saved authorization token.


In [228]:
island_info = retrieve_island_info('Dhakandhoo', 'Maldives', verbose=False)

ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['coastline_position_transect_2_waterline']
ts_we = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['wave_energy_of_combined_wind_waves_and_swell']
ts_sl = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['sea_level_anomaly']
ts_sst = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['sea_surface_temperature']
ts_tp = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['total_precipitation']
ts_temp = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['2_metre_dewpoint_temperature']
ts_wd = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['mean_wave_direction']
ts_windd = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['wind_direction_10m']
ts_winds = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['wind_speed_10m']
ts_ap = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_2_waterline']['monthly']['mean_sea_level_pressure']

import Rbeast as rb

o = rb.beast(ts, start=[ts.index[0].year, ts.index[0].month, ts.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_we = rb.beast(ts_we, start=[ts_we.index[0].year, ts_we.index[0].month, ts_we.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_sl = rb.beast(ts_sl, start=[ts_sl.index[0].year, ts_sl.index[0].month, ts_sl.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_sst = rb.beast(ts_sst, start=[ts_sst.index[0].year, ts_sst.index[0].month, ts_sst.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_tp = rb.beast(ts_tp, start=[ts_tp.index[0].year, ts_tp.index[0].month, ts_tp.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_temp = rb.beast(ts_temp, start=[ts_temp.index[0].year, ts_temp.index[0].month, ts_temp.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_wd = rb.beast(ts_wd, start=[ts_wd.index[0].year, ts_wd.index[0].month, ts_wd.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_windd = rb.beast(ts_windd, start=[ts_windd.index[0].year, ts_windd.index[0].month, ts_windd.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_winds = rb.beast(ts_winds, start=[ts_winds.index[0].year, ts_winds.index[0].month, ts_winds.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
o_ap = rb.beast(ts_ap, start=[ts_ap.index[0].year, ts_ap.index[0].month, ts_ap.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
# o_wd = rb.beast(ts_wd, start=[ts_wd.index[0].year, ts_wd.index[0].month, ts_wd.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)

names = ['coastline_position', 'wave_energy', 'sea_level_anomaly', 'sst', 'total_precipitation', 'dewpoint_temperature', 'mean_wave_direction', 'wind_direction_10m', 'wind_speed_10m', 'mean_sea_level_pressure']
o_list = [o, o_we, o_sl, o_sst, o_tp, o_temp, o_wd, o_windd, o_winds, o_ap]
ts_list = [ts, ts_we, ts_sl, ts_sst, ts_tp, ts_temp, ts_wd, ts_windd, ts_winds, ts_ap]
ts_dict, ts_dict_trend, ts_dict_seasonal, ts_dict_res = {}, {}, {}, {}
for ts, o, name in zip(ts_list, o_list, names):
    ts_dict[name] = ts
    ts_dict_trend[name] = o.trend.Y
    ts_dict_seasonal[name] = o.season.Y
    ts_dict_res[name] = ts - o.trend.Y - o.season.Y


# ts_dict = {'wave_energy': ts_we, 'coastline_position': ts, 'sea_level_anomaly': ts_sl, 'sst': ts_sst} #, 'mean_wave_direction': ts_wd}
# ts_dict_trend = {'wave_energy': o_we.trend.Y, 'coastline_position': o.trend.Y, 'sea_level_anomaly': o_sl.trend.Y, 'sst': o_sst.trend.Y} #, 'mean_wave_direction': o_wd.trend.Y}
# ts_dict_seasonal = {'wave_energy': o_we.season.Y, 'coastline_position': o.season.Y, 'sea_level_anomaly': o_sl.season.Y, 'sst': o_sst.season.Y} #, 'mean_wave_direction': o_wd.season.Y}
# ts_dict_res = {'wave_energy': ts_we - o_we.trend.Y - o_we.season.Y, 'coastline_position': ts - o.trend.Y - o.season.Y, 'sea_level_anomaly': ts_sl - o_sl.trend.Y - o_sl.season.Y, 'sst': ts_sst - o_sst.trend.Y - o_sst.season.Y}#, 'mean_wave_direction': ts_wd - o_wd.trend.Y - o_wd.season.Y}

ts_df = pd.DataFrame(ts_dict)
ts_df_trend = pd.DataFrame(ts_dict_trend)
ts_df_seasonal = pd.DataFrame(ts_dict_seasonal)
ts_df_res = pd.DataFrame(ts_dict_res)


In [161]:
dictt = TimeSeriesConnections(ts_dict).main()


-------------------------------------------------------------------
Evaluating time series connections
-------------------------------------------------------------------

Evaluating connections between time series: wave_energy & coastline_position
--- Evaluating correlation at lag 0 ---
--- Evaluating cross-correlation ---
--- Evaluating correlation at lag 4 ---
--- Evaluating correlation at lag -2 ---
--- Evaluating cointegration ---
No cointegration exists between time series
Evaluating connections between time series: wave_energy & sea_level_anomaly
--- Evaluating correlation at lag 0 ---
--- Evaluating cross-correlation ---
--- Evaluating correlation at lag 4 ---
--- Evaluating correlation at lag -3 ---
--- Evaluating cointegration ---
Cointegration exists between time series
Evaluating connections between time series: wave_energy & sst
--- Evaluating correlation at lag 0 ---
--- Evaluating cross-correlation ---


UnboundLocalError: cannot access local variable 'first_lag_significant_pos' where it is not associated with a value

In [15]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax2 = ax.twinx()
ts1.plot(ax=ax, color='r')
ts2.plot(ax=ax2, color='b')

<Axes: >

In [5]:
from statsmodels.tsa.stattools import ccf, grangercausalitytests, coint, adfuller

t_statistic, p_val, critical_p_val = coint(ts1, ts2, trend='ct', autolag=None, maxlag=3)
print(f' t statistic: {np.round(t_statistic, 2)} \n p value: {np.round(p_val,2)} \n critical p values [1%, 5%, 10%] {critical_p_val}')




 t statistic: -3.83 
 p value: 0.04 
 critical p values [1%, 5%, 10%] [-4.50158458 -3.88654012 -3.57500991]


In [6]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# 4. Johansen Co-integration Test
data = pd.concat([ts1, ts2], axis=1)
result = coint_johansen(data, det_order=0, k_ar_diff=3)

print('Johansen Test Statistics:')
print(result.lr1)  # Trace statistic
print('Critical Values (90%, 95%, 99%):')
print(result.cvt)

# Interpret the Johansen test results
for i in range(len(result.lr1)):
    trace_stat = result.lr1[i]
    critical_values = result.cvt[i]
    print(f'Trace statistic for r<={i}: {trace_stat:.2f}, critical values {critical_values}')

    if trace_stat > critical_values[1]:  # Compare to 95% critical value
        print(f'Null hypothesis of r<={i} is rejected at the 95% confidence level.')
    else:
        print(f'Null hypothesis of r<={i} cannot be rejected at the 95% confidence level.')

Johansen Test Statistics:
[56.70130768 11.12405127]
Critical Values (90%, 95%, 99%):
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
Trace statistic for r<=0: 56.70, critical values [13.4294 15.4943 19.9349]
Null hypothesis of r<=0 is rejected at the 95% confidence level.
Trace statistic for r<=1: 11.12, critical values [2.7055 3.8415 6.6349]
Null hypothesis of r<=1 is rejected at the 95% confidence level.


In [7]:
# Extract co-integrating vectors and compute the co-integrating relationship
coint_vectors = result.evec
coint_vector = coint_vectors[:, 0]  # First co-integrating vector

# data = pd.concat([ts1, ts2], axis=1)

# Calculate the co-integrating relationship
coint_relationship = np.dot(data_s, coint_vector)

# Plot the time series and co-integrating relationship on the same plot
plt.figure(figsize=(14, 6))
plt.plot(ts1, label='Time Series 1', color='blue')
plt.plot(ts2, label='Time Series 2', color='red')
plt.plot(data.index, coint_relationship, label='Co-integrating Relationship', color='green', linestyle='--')
plt.legend()
plt.title('Time Series and Co-integrating Relationship')
plt.show()

# Test for stationarity of the residuals
residuals = coint_relationship - np.mean(coint_relationship)
adf_result = adfuller(residuals)
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')
print(f'Critical Values: {adf_result[4]}')

# Plot the residuals of the co-integrating relationship on a separate plot
plt.figure(figsize=(14, 6))
plt.plot(data.index, residuals, label='Residuals of Co-integrating Relationship', color='purple')
plt.axhline(0, color='black', linestyle='--')
plt.title('Residuals of Co-integrating Relationship')
plt.legend()
plt.show()

NameError: name 'data_s' is not defined

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# import plot_acf
from statsmodels.graphics.tsaplots import plot_acf

import Rbeast as rb

# Plot original time series
plt.figure(figsize=(14, 6))
plt.plot(data['coastline_position_transect_2_waterline'], label='Time Series 1')
plt.plot(data['wave_energy_of_combined_wind_waves_and_swell'], label='Time Series 2')
plt.legend()
plt.title('Original Time Series with Seasonality')
plt.show()

# Seasonal Decomposition
# decomposition_ts1 = seasonal_decompose(data['coastline_position_transect_2_waterline'], model='additive', period=12)
# decomposition_ts2 = seasonal_decompose(data['wave_energy_of_combined_wind_waves_and_swell'], model='additive', period=12)
decomposition_ts1 = rb.beast(data['coastline_position_transect_2_waterline'], period='1 year', deltat='1/12 year')
decomposition_ts2 = rb.beast(data['wave_energy_of_combined_wind_waves_and_swell'], period='1 year', deltat='1/12 year')
rb.plot(decomposition_ts2)

plot_acf(data['coastline_position_transect_2_waterline'])

# # Plot seasonal decomposition
# plt.figure(figsize=(14, 12))

# plt.subplot(2, 3, 1)
# plt.plot(data['coastline_position_transect_2_waterline'], label='Time Series 1')
# plt.legend()
# plt.title('Time Series 1')

# plt.subplot(2, 3, 2)
# plt.plot(decomposition_ts1.trend.dropna(), label='Trend Component')
# plt.legend()
# plt.title('Trend Component of TS1')

# plt.subplot(2, 3, 3)
# plt.plot(decomposition_ts1.seasonal, label='Seasonal Component')
# plt.legend()
# plt.title('Seasonal Component of TS1')

# plt.subplot(2, 3, 4)
# plt.plot(data['wave_energy_of_combined_wind_waves_and_swell'], label='Time Series 2')
# plt.legend()
# plt.title('Time Series 2')

# plt.subplot(2, 3, 5)
# plt.plot(decomposition_ts2.trend.dropna(), label='Trend Component')
# plt.legend()
# plt.title('Trend Component of TS2')

# plt.subplot(2, 3, 6)
# plt.plot(decomposition_ts2.seasonal, label='Seasonal Component')
# plt.legend()
# plt.title('Seasonal Component of TS2')

# plt.tight_layout()
# plt.show()

# # Remove seasonality by subtracting seasonal component
# data_deseasonalized = pd.DataFrame({
#     'ts1_deseasonalized': data['coastline_position_transect_2_waterline'] - decomposition_ts1.seasonal,
#     'ts2_deseasonalized': data['wave_energy_of_combined_wind_waves_and_swell'] - decomposition_ts2.seasonal
# })

# # Perform Johansen test on deseasonalized data
# result_deseasonalized = coint_johansen(data_deseasonalized, det_order=0, k_ar_diff=3)

# # Extract co-integrating vectors and compute the co-integrating relationship
# coint_vectors_deseasonalized = result_deseasonalized.evec
# coint_vector_deseasonalized = coint_vectors_deseasonalized[:, 0]  # First co-integrating vector

# # Calculate the co-integrating relationship
# coint_relationship_deseasonalized = np.dot(data_deseasonalized, coint_vector_deseasonalized)

# # Plot deseasonalized time series and co-integrating relationship
# plt.figure(figsize=(14, 6))
# # plt.plot(data_deseasonalized['ts1_deseasonalized'], label='Deseasonalized Time Series 1', color='blue')
# # plt.plot(data_deseasonalized['ts2_deseasonalized'], label='Deseasonalized Time Series 2', color='red')
# plt.plot(data.index, coint_relationship_deseasonalized, label='Co-integrating Relationship', color='green', linestyle='--')
# plt.legend()
# plt.title('Deseasonalized Time Series and Co-integrating Relationship')
# plt.show()


INFO: To supress printing the parameers in beast(),      set print.options = 0 
INFO: To supress printing the parameers in beast_irreg(),set print.options = 0 
INFO: To supress printing the parameers in beast123(),   set extra.printOptions = 0  
INFO: To supress warning messages in beast(),            set quiet = 1 
INFO: To supress warning messages in beast_irreg(),      set quiet = 1 
INFO: To supress warning messages in beast123(),         set extra.quiet = 1  

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 92
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  0.0833333 year = 1 months = 30.4167 days
HasSeasonCmpnt: True  | period = 1 year = 12 months = 365 days. The model 'Y=Trend+Season+Error' is fitted.
              : Num_of_DataPoints_per_Period = period/deltaTime = 1/0.0833333 = 12
HasOutlier

<Figure size 640x480 with 1 Axes>

In [134]:
data_s = pd.DataFrame({'ts1_s': decomposition_ts1.season.Y, 'ts2_s': decomposition_ts2.season.Y}, index=ts1.index)
data_res = pd.DataFrame({'ts1_res': ts1 - decomposition_ts1.trend.Y - decomposition_ts1.season.Y, 'ts2_res': ts2 - decomposition_ts2.trend.Y - decomposition_ts2.season.Y}, index=ts1.index)
data_t = pd.DataFrame({'ts1_t': decomposition_ts1.trend.Y, 'ts2_t': decomposition_ts2.trend.Y}, index=ts1.index)

In [143]:
fig, ax = plt.subplots()
ax.plot(ts1.index, decomposition_ts2.trend.Y)
ax2 = ax.twinx()
ax2.plot(ts1.index, decomposition_ts1.trend.Y, color='r')

In [145]:
rb.plot(decomposition_ts2)

(<Figure size 640x480 with 9 Axes>,
 array([<Axes: xlabel='[]', ylabel='Y'>,
        <Axes: xlabel='[]', ylabel='season'>,
        <Axes: xlabel='[]', ylabel='Pr(scp)'>,
        <Axes: xlabel='[]', ylabel='sOrder'>,
        <Axes: xlabel='[]', ylabel='trend'>,
        <Axes: xlabel='[]', ylabel='Pr(tcp)'>,
        <Axes: xlabel='[]', ylabel='tOrder'>,
        <Axes: xlabel='[]', ylabel='slpsgn'>,
        <Axes: xlabel='time', ylabel='error'>], dtype=object))

In [141]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# 4. Johansen Co-integration Test
result = coint_johansen(data_t, det_order=0, k_ar_diff=1)

print('Johansen Test Statistics:')
print(result.lr1)  # Trace statistic
print('Critical Values (90%, 95%, 99%):')
print(result.cvt)


# Interpret the Johansen test results
for i in range(len(result.lr1)):
    trace_stat = result.lr1[i]
    critical_values = result.cvt[i]
    print(f'Trace statistic for r<={i}: {trace_stat:.2f}, critical values {critical_values}')

    if trace_stat > critical_values[1]:  # Compare to 95% critical value
        print(f'Null hypothesis of r<={i} is rejected at the 95% confidence level.')
    else:
        print(f'Null hypothesis of r<={i} cannot be rejected at the 95% confidence level.')


Johansen Test Statistics:
[10.22817199  2.17772393]
Critical Values (90%, 95%, 99%):
[[13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]]
Trace statistic for r<=0: 10.23, critical values [13.4294 15.4943 19.9349]
Null hypothesis of r<=0 cannot be rejected at the 95% confidence level.
Trace statistic for r<=1: 2.18, critical values [2.7055 3.8415 6.6349]
Null hypothesis of r<=1 cannot be rejected at the 95% confidence level.


# Causal inference and discovery

In [36]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib qt    
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
import sklearn

import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys

from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.lpcmci import LPCMCI

from tigramite.independence_tests.parcorr import ParCorr
from tigramite.independence_tests.robust_parcorr import RobustParCorr
from tigramite.independence_tests.parcorr_wls import ParCorrWLS 
from tigramite.independence_tests.gpdc import GPDC
from tigramite.independence_tests.cmiknn import CMIknn
from tigramite.independence_tests.cmisymb import CMIsymb
from tigramite.independence_tests.gsquared import Gsquared
from tigramite.independence_tests.regressionCI import RegressionCI

In [229]:
# Initialize dataframe object, specify time axis and variable names
var_names = names
dataframe = pp.DataFrame(ts_df.values, 
                         datatime = ts_df.index, 
                         var_names=var_names)

tp.plot_timeseries(dataframe)

(<Figure size 640x480 with 10 Axes>,
 array([<Axes: ylabel='coastline_position'>, <Axes: ylabel='wave_energy'>,
        <Axes: ylabel='sea_level_anomaly'>, <Axes: ylabel='sst'>,
        <Axes: ylabel='total_precipitation'>,
        <Axes: ylabel='dewpoint_temperature'>,
        <Axes: ylabel='mean_wave_direction'>,
        <Axes: ylabel='wind_direction_10m'>,
        <Axes: ylabel='wind_speed_10m'>,
        <Axes: ylabel='mean_sea_level_pressure'>], dtype=object))

In [211]:
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=parcorr,
    verbosity=1)
correlations = pcmci.get_lagged_dependencies(tau_max=13, val_only=True)['val_matrix']


##
## Estimating lagged dependencies 
##

Parameters:

independence test = par_corr
tau_min = 0
tau_max = 13


In [192]:
matrix_lags = np.argmax(np.abs(correlations), axis=2)
tp.plot_scatterplots(dataframe=dataframe, add_scatterplot_args={'matrix_lags':matrix_lags}); plt.show()

In [193]:
tp.plot_densityplots(dataframe=dataframe, add_densityplot_args={'matrix_lags':matrix_lags})
plt.show()

In [212]:
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=parcorr,
    verbosity=1)

correlations = pcmci.get_lagged_dependencies(tau_max=13, val_only=True)['val_matrix']
lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names, 
                                    'x_base':5, 'y_base':.5}); plt.show()


##
## Estimating lagged dependencies 
##

Parameters:

independence test = par_corr
tau_min = 0
tau_max = 13


In [221]:
tau_max = 12
pc_alpha = 0.01
pcmci.verbosity = 1
results = pcmci.run_pcmci(tau_max=tau_max, pc_alpha=None, alpha_level=0.1)


##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 12
pc_alpha = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
max_conds_dim = None
max_combinations = 1



## Resulting lagged parent (super)sets:

    Variable coastline_position has 10 link(s):
    [pc_alpha = 0.5]
        (coastline_position -1): max_pval = 0.00249, |min_val| =  0.393
        (mean_wave_direction -1): max_pval = 0.05515, |min_val| =  0.250
        (mean_wave_direction -2): max_pval = 0.19077, |min_val| =  0.164
        (total_precipitation -12): max_pval = 0.22805, |min_val| =  0.153
        (wind_direction_10m -5): max_pval = 0.22736, |min_val| =  0.148
        (mean_wave_direction -10): max_pval = 0.24553, |min_val| =  0.147
        (coastline_position -4): max_pval = 0.42312, |min_val| =  0.099
        (wind_direction_10m -6): max_pval = 0.45706, |min_val| =  0.098
        (coastline_position -2): max_pval = 0.48467, |min_val| =  0.087
        (wave_ene

In [224]:
tau_max = 12
pc_alpha = 0.1
pcmci.verbosity = 1

results = pcmci.run_pcmciplus(tau_min=0, tau_max=tau_max, pc_alpha=pc_alpha)


##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 12
pc_alpha = [0.1]
max_conds_dim = None
max_combinations = 1



## Resulting lagged parent (super)sets:

    Variable coastline_position has 3 link(s):
        (coastline_position -1): max_pval = 0.00000, |min_val| =  0.646
        (mean_wave_direction -1): max_pval = 0.00284, |min_val| =  0.362
        (total_precipitation -12): max_pval = 0.08332, |min_val| =  0.215

    Variable wave_energy has 5 link(s):
        (wave_energy -12): max_pval = 0.00010, |min_val| =  0.467
        (sst -2): max_pval = 0.00712, |min_val| =  0.331
        (mean_wave_direction -6): max_pval = 0.07268, |min_val| =  0.226
        (coastline_position -12): max_pval = 0.09524, |min_val| =  0.210
        (coastline_position -5): max_pval = 0.09931, |min_val| =  0.206

    Variable sea_level_anomaly has 5 link(s):
        (sea_level_anomaly -1): max_pval = 0.00003, |min_val| =  0.483


In [226]:
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=tau_max, fdr_method='fdr_bh')
# pcmci.print_significant_links(
#         p_matrix = q_matrix,
#         val_matrix = results['val_matrix'],
#         alpha_level = 0.1)
graph = pcmci.get_graph_from_pmatrix(p_matrix=q_matrix, alpha_level=0.1, 
            tau_min=0, tau_max=tau_max, link_assumptions=None)
results['graph'] = graph

In [227]:
tp.plot_graph(
    val_matrix=results['val_matrix'],
    graph=results['graph'],
    var_names=var_names,
    link_colorbar_label='cross-MCI (edges)',
    node_colorbar_label='auto-MCI (nodes)',
    ); plt.show()

In [148]:
# Plot time series graph    
tp.plot_time_series_graph(
    figsize=(6, 4),
    val_matrix=results['val_matrix'],
    graph=results['graph'],
    var_names=var_names,
    link_colorbar_label='MCI',
    ); plt.show()